# Process raw model output

This notebook processes the raw model output from `MITgcm-Michigan-phosphorus`. 

**Raw files:** The raw output files are publicly available [here](https://figshare.com/account/home#/projects/37949)
* model grid
* Daily simulated surface total phosphorus (TP) for 2007-2010
* Ecosystem service (ES) values shapefile

**Metrics calculated:**
* Normalized ES value : $DT=\log_{10}(ES+1)$ then $\frac{DT - min(DT)}{max(DT) - min(DT)}$
* Exceedance days (ED) : number of days TP exceeds 0.4
* Cumulative stress days (CSD) : total number of days all services are stresed

**Processed files location** 
* `/Users/gloege/Documents/Projects/MITgcm-Michigan-Phosphorus/data/clean/`

In [1]:
import numpy as np
import xarray as xr
import xshape
import pandas as pd
import sys
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
print(sys.version)
print('-----------------------------------')
print(f'numpy version  : {np.__version__}')
print(f'xarray version : {xr.__version__}')
print(f'pandas version : {pd.__version__}')
print(f'xshape version : {xshape.__version__}')

3.6.6 |Anaconda, Inc.| (default, Jun 28 2018, 11:07:29) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
-----------------------------------
numpy version  : 1.14.3
xarray version : 0.10.8
pandas version : 0.23.4
xshape version : 0.1.1


# 0. Define functions

In [3]:
dir_notebook='/Users/gloege/Documents/Projects/MITgcm-Michigan-Phosphorus/scripts/'
%run {dir_notebook}_define_functions.ipynb

# 1. phosphorus_max_JJA.nc
1. Load raw surface concentration (daily averages 2007-2010)
2. Calculate cumulative concentration
3. Change time vector
4. Calculate maximum across JJA in a "typical" year. Typical is defined as the the DOY average across 2007-2010
5. Save to dataset ds_out and display
6. save to netcdf and delete ds_out from memory

In [4]:
### =================================================
### 1. Open datasets (open_mfdataset opens all globbed files)   
### =================================================
dir_raw = '/Users/gloege/Documents/Projects/MITgcm-Michigan-Phosphorus/data/raw/'
ds = xr.open_mfdataset(f'{dir_raw}ptr_tave.*.surf.nc')
ds_grid = xr.open_dataset(f'{dir_raw}grid_lake_michigan.nc')

### =================================================
### 2. Calculate cumulative concentration
### =================================================
ds['cumulative'] = ds['Fox'] + ds['Grand'] + ds['Kalamazoo'] + ds['Manistee'] + \
                   ds['Manistique'] + ds['Menominee'] + ds['Milwaukee'] + ds['Muskegon'] + \
                   ds['PereMarquette'] + ds['Sheboygan'] + ds['StJoseph']

### =================================================
### 3. Change time from model time to something
###    more intelligent
### =================================================
ds['T'] = pd.date_range(start='2007-01-01', end='2010-12-31', freq='1d')

### =================================================
### 4. Maximum summer concentration in a typical year
### 4.1 first pick out summer months (JJA) and drop other months
### 4.2 then group data by day-of-year and average all day-of-year
### 4.3 finally take max across observed at each location in a typical summer
### 4.4 squeeze just removes the singleton Z dimension. 
###     This is surface data
### =================================================
ds_jja = ds['cumulative'].where((ds['T.month']==6) | 
                                (ds['T.month']==7) | 
                                (ds['T.month']==8), drop=True)\
                                                    .groupby('T.dayofyear').mean('T')\
                                                    .max('dayofyear').squeeze()
        
### =================================================
### 5. Save to dataset
### =================================================
ds_out = xr.Dataset(
    {
    'phos_max_JJA': (['Y', 'X'],  ds_jja.values),
    },

    coords={
    'Y': (['Y'], ds_grid['Y']),
    'X': (['X'], ds_grid['X']),   
    }
    )

### =================================================
### 5.1 Display the dataset
### =================================================
ds_out

<xarray.Dataset>
Dimensions:       (X: 200, Y: 276)
Coordinates:
  * Y             (Y) float64 41.57 41.59 41.6 41.62 41.64 41.65 41.67 41.69 ...
  * X             (X) float64 -88.09 -88.07 -88.06 -88.04 -88.02 -88.01 ...
Data variables:
    phos_max_JJA  (Y, X) float32 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 ...

In [5]:
### =================================================
### 6. Save to netcdf
### =================================================
print('save to netcdf')
data_clean='/Users/gloege/Documents/Projects/MITgcm-Michigan-Phosphorus/data/clean/'
ds_out.to_netcdf(f'{data_clean}phosphorus_max_JJA.nc')

### =================================================
### 6.1 Delete ds_out from memory
### =================================================
del ds, ds_out, ds_grid, ds_jja

save to netcdf


# 1. Ecosystem service data
Three services are defined
1.  water = municipal water withdrawals
2. beaches = photo user days at beaches
3. boating = linear sum of marina slips and boat launches to define a single "boating" metric. This is not perfect becuase marinas and boat launches cater to difference demagraphics

Steps to process ES data
1. load grid and ES data
2. normalize each service
3. save dataset
4. save netcdf

In [6]:
### =================================================
### 1. Load grid
### =================================================
grid_dir = '/Users/gloege/Documents/Projects/MITgcm-Michigan-Phosphorus/data/raw/'
ds_grid = xr.open_dataset(f'{grid_dir}grid_lake_michigan.nc')

### =================================================
### 1.1 Load ES shapefiles
### =================================================
data_dir = '/Users/gloege/Documents/Projects/MITgcm-Michigan-Phosphorus/data/shapefiles/'
ES_shp = f'{data_dir}grid_WGS84_ES/grid_WGS84_ES/grid_WGS84_ES.shp'
fields, polygons = xshape.parse_shapefile(ES_shp)

### =================================================
### 2. Normalize and reshape services 
###    I defined a 'normalize' function
### =================================================
beaches_norm    = normalize( np.flipud(np.reshape(fields['FLICKR'].values, (200,276)).T) )
water_norm      = normalize( np.flipud(np.reshape(fields['WATWITHDWL'].values, (200,276)).T) )
marina_norm     = normalize( np.flipud(np.reshape(fields['MARSLIPS'].values, (200,276)).T) )
boatlaunch_norm = normalize( np.flipud(np.reshape(fields['BOATLPARK'].values, (200,276)).T) )
boating_norm    = normalize( np.flipud(np.reshape(fields['BOATLPARK'].values + \
                                                  fields['MARSLIPS'].values, (200,276)).T) )

### =================================================
### 3. Save to dataset
### =================================================
ds_out = xr.Dataset(
    {
    'beaches_norm': (['Y', 'X'],  beaches_norm),
    'water_norm': (['Y', 'X'],  water_norm),
    'boating_norm': (['Y', 'X'],  boating_norm),
    'marina_norm': (['Y', 'X'],  marina_norm),
    'boatlaunch_norm': (['Y', 'X'],  boatlaunch_norm)
    },

    coords={
    'Y': (['Y'], ds_grid['Y']),
    'X': (['X'], ds_grid['X']),   
    }
    )

### =================================================
### 3.1 Display the dataset
### =================================================
ds_out

<xarray.Dataset>
Dimensions:          (X: 200, Y: 276)
Coordinates:
  * Y                (Y) float64 41.57 41.59 41.6 41.62 41.64 41.65 41.67 ...
  * X                (X) float64 -88.09 -88.07 -88.06 -88.04 -88.02 -88.01 ...
Data variables:
    beaches_norm     (Y, X) float64 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 ...
    water_norm       (Y, X) float64 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 ...
    boating_norm     (Y, X) float64 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 ...
    marina_norm      (Y, X) float64 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 ...
    boatlaunch_norm  (Y, X) float64 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 ...

In [7]:
### =================================================
### 4. save to netcdf
### =================================================
print('save to netcdf')
data_clean='/Users/gloege/Documents/Projects/MITgcm-Michigan-Phosphorus/data/clean/'
ds_out.to_netcdf(f'{data_clean}normalized_es.nc')

### =================================================
### 4.1 Delete ds_out from memory
### =================================================
del ds_out, ds_grid
del ES_shp, fields, polygons
del beaches_norm, water_norm, marina_norm, boatlaunch_norm, boating_norm

save to netcdf


# 2. Calculate an average May June July August 

1. open all files `/Users/gloege/Documents/Projects/lakeMichigan/data/output_phos/ptr_tave.*.surf.nc` 
2. create regions (cumulative, north, east, west)
3. change time variable from model time to YYYY-MM-DD
4. select May, June, July, August for all years. Groupby day-of-year and average

In [8]:
### ===================================
### 1. Open dataset
### ===================================
### the `squeeze()` command ignores the the `z` dimension, since this is just surface data
data_dir = '/Users/gloege/Documents/Projects/MITgcm-Michigan-Phosphorus/data/raw/'
ds = xr.open_mfdataset(f'{data_dir}ptr_tave.*.surf.nc').squeeze()

### ===================================
### 2. create regions
### ===================================
### Cumulative of all rivers
ds['cumulative'] = ds['Fox'] + ds['Grand'] + ds['Kalamazoo'] + ds['Manistee'] + \
                   ds['Manistique'] + ds['Menominee'] + ds['Milwaukee'] + ds['Muskegon'] + \
                   ds['PereMarquette'] + ds['Sheboygan'] + ds['StJoseph']

### North region
ds['north'] = ds['Fox'] + ds['Menominee'] + ds['Manistique'] 
  
### East Region
ds['east'] =  ds['Grand'] + ds['Kalamazoo'] + ds['Manistee'] + \
              ds['Muskegon'] + ds['PereMarquette']  + ds['StJoseph']
        
### West region
ds['west'] = ds['Milwaukee'] + ds['Sheboygan']

### ===================================
### 3. Add a sensible time varaiable
### ===================================
ds['T'] = pd.date_range(start='2007-01-01', end='2010-12-31', freq='1d')

### ===================================
### 4. Average May, June, July, August
### ===================================
ds_mjja = ds.where((ds['T.month']==5) |
         (ds['T.month']==6) | 
         (ds['T.month']==7) | 
         (ds['T.month']==8), 
         drop = True).groupby('T.dayofyear').mean('T')

### ===================================
### 4.1. display dataset
### ===================================
ds_mjja

<xarray.Dataset>
Dimensions:        (X: 200, Y: 276, dayofyear: 124)
Coordinates:
  * X              (X) float64 -88.09 -88.07 -88.06 -88.04 -88.02 -88.01 ...
  * Y              (Y) float64 41.57 41.59 41.6 41.62 41.64 41.65 41.67 ...
    Z              float64 -2.5
  * dayofyear      (dayofyear) int64 121 122 123 124 125 126 127 128 129 130 ...
Data variables:
    Fox            (dayofyear, Y, X) float32 dask.array<shape=(124, 276, 200), chunksize=(1, 276, 200)>
    Grand          (dayofyear, Y, X) float32 dask.array<shape=(124, 276, 200), chunksize=(1, 276, 200)>
    Kalamazoo      (dayofyear, Y, X) float32 dask.array<shape=(124, 276, 200), chunksize=(1, 276, 200)>
    Manistee       (dayofyear, Y, X) float32 dask.array<shape=(124, 276, 200), chunksize=(1, 276, 200)>
    Manistique     (dayofyear, Y, X) float32 dask.array<shape=(124, 276, 200), chunksize=(1, 276, 200)>
    Menominee      (dayofyear, Y, X) float32 dask.array<shape=(124, 276, 200), chunksize=(1, 276, 200)>
    Milwauke

In [9]:
### ===================================
### 5. Save to netcdf
### ===================================
print('save to netcdf')
data_clean='/Users/gloege/Documents/Projects/MITgcm-Michigan-Phosphorus/data/clean/'
ds_mjja.to_netcdf(f'{data_clean}phosphorus_avg_MJJA.nc')

### ===================================
### 5.1 wipe memory
### ===================================
del ds, ds_mjja

save to netcdf


# 3. Calculate exceedance days for each river

1. load phosphorus_avg_mjja.nc data
2. Calculate exceedance days:
    - At each point ask "is the concentration >0.4?" and create a boolean
    - bin the data by month. Bin values correspond to the day-of-year
    - sum across each month
3. rename the groupby_bins to month
4. save data to netcdf

#### simple example to convince me groupy_bins().sum() is doing what I want
`ds_test = xr.Dataset({'data' : (['X'], [1,2,3,4,5])}, coords={'X': (['X'], [6,7,8,9,10])})`
`ds_test['data'].groupby_bins('X',[6,7,9], include_lowest=True).sum('X')`

In [10]:
### ===================================
### 1. Load average MJJA data
### ===================================
data_clean='/Users/gloege/Documents/Projects/MITgcm-Michigan-Phosphorus/data/clean/'
ds_mjja = xr.open_dataset(f'{data_clean}phosphorus_avg_MJJA.nc')

### ===================================
### 2. calculate exceedance days for each month
### ===================================
ds_out = (ds_mjja > 0.4).groupby_bins('dayofyear',
                                      bins = [121, 151, 181, 212, 243],
                                      labels = ('may', 'june','july','august'),
                                      include_lowest = True).sum('dayofyear')
### ===================================
### 3. Rename bin name
### ===================================
ds_out = ds_out.rename({'dayofyear_bins':'month'}) 

### ===================================
### 3.1 Display data
### ===================================
ds_out

<xarray.Dataset>
Dimensions:        (X: 200, Y: 276, month: 4)
Coordinates:
  * month          (month) object 'may' 'june' 'july' 'august'
  * X              (X) float64 -88.09 -88.07 -88.06 -88.04 -88.02 -88.01 ...
  * Y              (Y) float64 41.57 41.59 41.6 41.62 41.64 41.65 41.67 ...
    Z              float64 -2.5
Data variables:
    Fox            (month, Y, X) int64 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
    Grand          (month, Y, X) int64 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
    Kalamazoo      (month, Y, X) int64 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
    Manistee       (month, Y, X) int64 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
    Manistique     (month, Y, X) int64 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
    Menominee      (month, Y, X) int64 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
    Milwaukee      (month, Y, X) int64 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
    Muskegon       (month, Y, X) int64 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
    PereMarq

In [11]:
### ===================================
### 4. Save data to netcdf
### ===================================
print('save to netcdf')
data_clean='/Users/gloege/Documents/Projects/MITgcm-Michigan-Phosphorus/data/clean/'
ds_out.to_netcdf(f'{data_clean}exceedance_days_all_rivers_MJJA.nc')

### ===================================
### 4.1 Delete ds_out from memory
### ===================================
del ds_out, ds_mjja

save to netcdf


# 4. Calculate cumulative stress days (CSD)

0. csd_rm() is a helper function to calculate csd value for any given river and month
1. calculate CSD for each river and month and store in dataset
2. save data to netcdf

In [12]:
def csd_rm(river='cumulative', month='june'):
    '''
    csd_rm(river, month):
    this function calculates CSD for a given river and month
    You need to have a netcdf of ES and ED for this to work
    It's intended purpose is to make it easy to put all CSD data into
    an xarray dataset
    '''
    ### ===================================
    ### Clean data directory
    ### ===================================
    data_clean='/Users/gloege/Documents/Projects/MITgcm-Michigan-Phosphorus/data/clean/'
    ds_es = xr.open_dataset(f'{data_clean}normalized_es.nc')

    ### ===================================
    ### 1. Load ED data
    ### ===================================
    ds_ed = xr.open_dataset(f'{data_clean}exceedance_days_all_rivers_MJJA.nc')
    ds_ed = ds_ed.where(ds_ed['month']== month , drop=True).squeeze()

    ### ===================================
    ### 2. calculate CSD
    ### ===================================
    csd = calculate_csd(ds_ed[river].values, ds_es['beaches_norm'].values) + \
            calculate_csd(ds_ed[river].values, ds_es['water_norm'].values) + \
            calculate_csd(ds_ed[river].values, ds_es['boating_norm'].values)
    
    return csd

In [13]:
### ===================================
### 1. Calculate CSD for June, July, August
###    for each river using csd_rm()
###    helper function
### ===================================
ds_out = xr.Dataset(
    {
    'Milwaukee': (['months'],  [csd_rm(river='Milwaukee', month='june'), 
                                csd_rm(river='Milwaukee', month='july'), 
                                csd_rm(river='Milwaukee', month='august')]),
    'Sheboygan': (['months'],  [csd_rm(river='Sheboygan', month='june'), 
                                csd_rm(river='Sheboygan', month='july'), 
                                csd_rm(river='Sheboygan', month='august')]),
    'Fox': (['months'],        [csd_rm(river='Fox', month='june'), 
                                csd_rm(river='Fox', month='july'), 
                                csd_rm(river='Fox', month='august')]),
    'Menominee': (['months'],  [csd_rm(river='Menominee', month='june'), 
                                csd_rm(river='Menominee', month='july'), 
                                csd_rm(river='Menominee', month='august')]),
    'Manistique': (['months'],  [csd_rm(river='Manistique', month='june'), 
                                csd_rm(river='Manistique', month='july'), 
                                csd_rm(river='Manistique', month='august')]),
    'Manistee': (['months'],   [csd_rm(river='Manistee', month='june'), 
                                csd_rm(river='Manistee', month='july'), 
                                csd_rm(river='Manistee', month='august')]),
    'Pere_Marquette': (['months'],  [csd_rm(river='PereMarquette', month='june'), 
                                csd_rm(river='PereMarquette', month='july'), 
                                csd_rm(river='PereMarquette', month='august')]),
    'Muskegon': (['months'],   [csd_rm(river='Muskegon', month='june'), 
                                csd_rm(river='Muskegon', month='july'), 
                                csd_rm(river='Muskegon', month='august')]),
    'Grand': (['months'],     [csd_rm(river='Grand', month='june'), 
                                csd_rm(river='Grand', month='july'), 
                                csd_rm(river='Grand', month='august')]),
    'Kalamazoo': (['months'],  [csd_rm(river='Kalamazoo', month='june'), 
                                csd_rm(river='Kalamazoo', month='july'), 
                                csd_rm(river='Kalamazoo', month='august')]),
    'St_Joseph': (['months'],  [csd_rm(river='StJoseph', month='june'), 
                                csd_rm(river='StJoseph', month='july'), 
                                csd_rm(river='StJoseph', month='august')]),
    'north': (['months'],      [csd_rm(river='north', month='june'), 
                                csd_rm(river='north', month='july'), 
                                csd_rm(river='north', month='august')]),
    'east': (['months'],       [csd_rm(river='east', month='june'), 
                                csd_rm(river='east', month='july'), 
                                csd_rm(river='east', month='august')]),
    'west': (['months'],      [csd_rm(river='west', month='june'), 
                                csd_rm(river='west', month='july'), 
                                csd_rm(river='west', month='august')]),
    'cumulative': (['months'],[csd_rm(river='cumulative', month='june'), 
                                csd_rm(river='cumulative', month='july'), 
                                csd_rm(river='cumulative', month='august')]),
    },

    coords={
    'months': (['months'], ['june', 'july', 'august'])
    }
    )

### ===================================
### 1.1 Display data
### ===================================
ds_out

<xarray.Dataset>
Dimensions:         (months: 3)
Coordinates:
  * months          (months) <U6 'june' 'july' 'august'
Data variables:
    Milwaukee       (months) float64 105.2 102.8 69.35
    Sheboygan       (months) float64 95.11 27.37 0.8099
    Fox             (months) float64 394.4 616.5 644.6
    Menominee       (months) float64 78.7 71.18 44.95
    Manistique      (months) float64 7.109 5.53 3.16
    Manistee        (months) float64 25.13 12.41 5.173
    Pere_Marquette  (months) float64 46.93 3.911 0.0
    Muskegon        (months) float64 3.595 0.0 0.2568
    Grand           (months) float64 331.4 323.6 222.6
    Kalamazoo       (months) float64 274.6 273.8 221.8
    St_Joseph       (months) float64 263.8 243.0 210.3
    north           (months) float64 509.0 674.0 668.9
    east            (months) float64 1.345e+03 1.014e+03 873.9
    west            (months) float64 207.5 136.0 76.23
    cumulative      (months) float64 2.068e+03 1.837e+03 1.644e+03

In [14]:
### ===================================
### 2. Save data to netcdf
### ===================================
print('save to netcdf')
data_clean='/Users/gloege/Documents/Projects/MITgcm-Michigan-Phosphorus/data/clean/'
ds_out.to_netcdf(f'{data_clean}cumulative_stress_days_all_rivers_JJA.nc')

### ===================================
### 2. Delete ds_out from memory
### ===================================
del ds_out

save to netcdf


# 4.1 calculate exceedance days (ED) shifted to service sites
1. load data
2. shift ED to ES sites for each month and store in dataset
3. save to netcdf

In [15]:
def shift_rm(river='cumulative'):
    '''
    csd_rm(river, month):
    this function calculates CSD for a given river and month
    You need to have a netcdf of ES and ED for this to work
    It's intended purpose is to make it easy to put all CSD data into
    an xarray dataset
    '''
    ### ===================================
    ### Clean data directory
    ### ===================================
    data_clean='/Users/gloege/Documents/Projects/MITgcm-Michigan-Phosphorus/data/clean/'
    
    ### ===================================
    ### create instance of ecosystem service class
    ### ===================================
    es = ecosystem_services()

    ### ===================================
    ### Load ES data
    ### ===================================
    ds_es = xr.open_dataset(f'{data_clean}normalized_es.nc')

    ### ===================================
    ### Load ED data
    ### ===================================
    ds_ed = xr.open_dataset(f'{data_clean}exceedance_days_all_rivers_MJJA.nc')

    ### ===================================
    ### sum the services to create a map of service sites
    ### The values do not matter. 
    ### I just need to know where service sites are
    ### ===================================
    service_locations = ds_es['beaches_norm']+ds_es['boating_norm']+ds_es['water_norm']

    ### ===================================
    ### Shift ED to the service sites for each month
    ### This uses a function in the es class
    ### ===================================
    june=es.shift_to_service(service_locations.values, 
                                   ds_ed[river].where(ds_ed['month']=='june', 
                                                      drop=True).squeeze())

    july=es.shift_to_service(service_locations.values, 
                                   ds_ed[river].where(ds_ed['month']=='july', 
                                                      drop=True).squeeze())

    august=es.shift_to_service(service_locations.values, 
                                   ds_ed[river].where(ds_ed['month']=='august', 
                                                      drop=True).squeeze())
    
    ### ===================================
    ### Stack all the months so the data has
    ### an output dimension like [month, X, Y]
    ### ===================================
    return np.stack((june, july, august))

In [16]:
### ===================================
### 1. Load grid values
### ===================================
data_raw='/Users/gloege/Documents/Projects/MITgcm-Michigan-Phosphorus/data/raw/'
ds_grid = xr.open_dataset(f'{data_raw}grid_lake_michigan.nc')

### ===================================
### 2. for each month, shift each river 
###    to service sites
### ===================================
ds_out = xr.Dataset(
    {
    'Milwaukee': (['months', 'Y', 'X'],  shift_rm(river='Milwaukee')),
    'Sheboygan': (['months', 'Y', 'X'],  shift_rm(river='Sheboygan')),
    'Fox': (['months', 'Y', 'X'],  shift_rm(river='Fox')),
    'Menominee': (['months',  'Y', 'X'], shift_rm(river='Menominee')),
    'Manistique': (['months',  'Y', 'X'], shift_rm(river='Manistique')),
    'Manistee': (['months',  'Y', 'X'], shift_rm(river='Manistee')),
    'Pere_Marquette': (['months',  'Y', 'X'], shift_rm(river='PereMarquette')),
    'Muskegon': (['months',  'Y', 'X'], shift_rm(river='Muskegon')),
    'Grand': (['months',  'Y', 'X'], shift_rm(river='Grand')),
    'Kalamazoo': (['months',  'Y', 'X'], shift_rm(river='Kalamazoo')),
    'St_Joseph': (['months',  'Y', 'X'], shift_rm(river='StJoseph')),
    'north': (['months',  'Y', 'X'], shift_rm(river='north')),
    'east': (['months',  'Y', 'X'], shift_rm(river='east')),
    'west': (['months',  'Y', 'X'], shift_rm(river='west')),
    'cumulative': (['months',  'Y', 'X'], shift_rm(river='cumulative')),
    },

    coords={
    'months': (['months'], ['june', 'july', 'august']),
    'Y': (['Y'], ds_grid['Y'].values),
    'X': (['X'], ds_grid['X'].values)
    }
    )

### ===================================
### 2.1 Display data
### ===================================
ds_out

<xarray.Dataset>
Dimensions:         (X: 200, Y: 276, months: 3)
Coordinates:
  * months          (months) <U6 'june' 'july' 'august'
  * Y               (Y) float64 41.57 41.59 41.6 41.62 41.64 41.65 41.67 ...
  * X               (X) float64 -88.09 -88.07 -88.06 -88.04 -88.02 -88.01 ...
Data variables:
    Milwaukee       (months, Y, X) float64 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 ...
    Sheboygan       (months, Y, X) float64 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 ...
    Fox             (months, Y, X) float64 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 ...
    Menominee       (months, Y, X) float64 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 ...
    Manistique      (months, Y, X) float64 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 ...
    Manistee        (months, Y, X) float64 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 ...
    Pere_Marquette  (months, Y, X) float64 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 ...
    Muskegon        (months, Y, X) float64 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 ...
    Grand           (months, Y, X) float64 0.0 0.0 0.0 0.0 0.0 

In [17]:
### ===================================
### 3. Save to netcdf
### ===================================
print('save to netcdf')
data_clean='/Users/gloege/Documents/Projects/MITgcm-Michigan-Phosphorus/data/clean/'
ds_out.to_netcdf(f'{data_clean}ED_at_ES_all_rivers_JJA.nc')

### ===================================
### 3.1 Delete ds_out from memory
### ===================================
del ds_out, ds_grid

save to netcdf


# 5. Calculate average concentration within the "footprint"

This averages the concentration within each month where concentration greater than 0.4

In [18]:
### ===================================
### 1. Load average MJJA data
### ===================================
data_clean='/Users/gloege/Documents/Projects/MITgcm-Michigan-Phosphorus/data/clean/'
ds_mjja = xr.open_dataset(f'{data_clean}phosphorus_avg_MJJA.nc')


### ===================================
### 2. find where concentration exceedes
###    the threshold concentration 0.4
###    and average the concentration within each month
### ===================================
ds_out = ds_mjja.where(ds_mjja > 0.4).groupby_bins('dayofyear',
                                      bins = [121, 151, 181, 212, 243],
                                      labels = ('may', 'june','july','august'),
                                      include_lowest = True).mean()
### ===================================
### 3. Rename bin name
### ===================================
ds_out = ds_out.rename({'dayofyear_bins':'month'}) 

### ===================================
### 3.1 Display dataset
### ===================================
ds_out

<xarray.Dataset>
Dimensions:        (month: 4)
Coordinates:
  * month          (month) object 'may' 'june' 'july' 'august'
    Z              float64 -2.5
Data variables:
    Fox            (month) float64 3.233 2.956 2.83 3.112
    Grand          (month) float64 0.7297 0.9559 0.7237 0.7182
    Kalamazoo      (month) float64 1.029 1.311 0.9851 1.03
    Manistee       (month) float64 0.546 0.6588 0.4402 0.4443
    Manistique     (month) float64 0.6859 0.7372 0.653 0.4578
    Menominee      (month) float64 0.6319 0.6813 0.6373 0.5888
    Milwaukee      (month) float64 0.7324 0.9127 0.9463 0.6838
    Muskegon       (month) float64 0.5702 0.6497 0.4441 0.4405
    PereMarquette  (month) float64 0.574 0.6575 0.4809 nan
    Sheboygan      (month) float64 0.5068 0.7888 0.739 0.5118
    StJoseph       (month) float64 0.6647 0.7629 0.6137 0.7953
    iter           (month) float64 6.117e+05 6.24e+05 6.372e+05 6.506e+05
    cumulative     (month) float64 1.519 1.478 1.946 2.44
    north          (

In [19]:
### ===================================
### 4. Save to netcdf
### ===================================
print('save to netcdf')
data_clean='/Users/gloege/Documents/Projects/MITgcm-Michigan-Phosphorus/data/clean/'
ds_out.to_netcdf(f'{data_clean}avg_TP_within_plume_all_rivers_MJJA.nc')

### ===================================
### 4.1 Delete ds_out from memory
### ===================================
del ds_out, ds_mjja

save to netcdf


# 6. calculate a transect on 43N
1. load raw 3D data
2. average across latitude band centered on 43N
3. store output in dataset
4. save to netcdf

In [20]:
### ==============================================================
### 1. Load raw 3D data
### ==============================================================
dir_raw='/Users/gloege/Documents/Projects/MITgcm-Michigan-Phosphorus/data/raw/'
ds_phos = xr.open_mfdataset(f'{dir_raw}output_3D/*pload*')
ds_phos['T'] = pd.date_range(start='2007-01-01', end='2010-12-31', freq='1d')
ds_temp = xr.open_mfdataset(f'{dir_raw}output_3D/*Ttave*')
ds_temp['T'] = pd.date_range(start='2007-01-01', end='2010-12-31', freq='1d')

### ==============================================================
### 2. Calculate transect
###.    average across latitude band centered on 43N
### ==============================================================
T_may = ds_temp.where((ds_temp['Y']>42.8) & (ds_temp['Y']<43.2), drop=True).\
                mean('Y').\
                where(ds_temp['T.month']==5, drop=True).mean('T')
        
T_august = ds_temp.where((ds_temp['Y']>42.8) & (ds_temp['Y']<43.2), drop=True).\
                mean('Y').\
                where(ds_temp['T.month']==8, drop=True).mean('T')
        
phos_may = ds_phos.where((ds_phos['Y']>42.8) & (ds_phos['Y']<43.2), drop=True).\
                mean('Y').\
                where(ds_phos['T.month']==5, drop=True).mean('T')
        
phos_august = ds_phos.where((ds_phos['Y']>42.8) & (ds_phos['Y']<43.2), drop=True).\
                mean('Y').\
                where(ds_phos['T.month']==8, drop=True).mean('T')

### ==============================================================
### 3. Save to dataset
### ==============================================================
ds_out = xr.Dataset(
    {
    'temp_may': (['Z', 'X'],  T_may['Ttave'].values),
    'temp_august': (['Z', 'X'],  T_august['Ttave'].values),
    'phos_may': (['Z', 'X'],  phos_may['pload'].values),
    'phos_august': (['Z', 'X'],  phos_august['pload'].values),
    },

    coords={
    'X': (['X'], ds_temp['X'].values),
    'Z': (['Z'], ds_temp['Z'].values),   
    }
    )

### ===================================
### 3.1 Display data
### ===================================
ds_out

<xarray.Dataset>
Dimensions:      (X: 200, Z: 29)
Coordinates:
  * X            (X) float64 -88.09 -88.07 -88.06 -88.04 -88.02 -88.01 ...
  * Z            (Z) float64 -2.5 -7.5 -12.5 -17.5 -22.5 -27.5 -32.5 -37.5 ...
Data variables:
    temp_may     (Z, X) float32 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 ...
    temp_august  (Z, X) float32 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 ...
    phos_may     (Z, X) float32 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 ...
    phos_august  (Z, X) float32 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 ...

In [21]:
### ==============================================================
### 4. save to netcdf
### ==============================================================
dir_clean='/Users/gloege/Documents/Projects/MITgcm-Michigan-Phosphorus/data/clean/'
ds_out.to_netcdf(f'{dir_clean}transect_43N.nc')

### ==============================================================
### 4.1 delete ds_out
### ==============================================================
del ds_out, ds_phos, ds_temp,
del T_may, T_august, phos_may, phos_august

# 7. Calculate monthly phosphorus concentration for each year
this is used to generate `supplemental_figure_2`

1. load data
2. create cumulative concentration by adding all the rivers
3. create a sensible time vector that is not model time
4. average concentration for each month and each year
5. stack all the years
6. store in dataset
7. store in netcdf

In [25]:
### ==============================================================
### 1. Load data
### ==============================================================
dir_raw = '/Users/gloege/Documents/Projects/MITgcm-Michigan-Phosphorus/data/raw/'
ds = xr.open_mfdataset(f'{dir_raw}ptr_tave.*.surf.nc').squeeze()
ds_grid = xr.open_dataset(f'{dir_raw}grid_lake_michigan.nc')

### ==============================================================
### 2. Create cumulative concentration
### ==============================================================
ds['cumulative'] = ds['Fox'] + ds['Grand'] + ds['Kalamazoo'] + ds['Manistee'] + \
                   ds['Manistique'] + ds['Menominee'] + ds['Milwaukee'] + ds['Muskegon'] + \
                   ds['PereMarquette'] + ds['Sheboygan'] + ds['StJoseph']

### ==============================================================
### 3. Add a sensible time varaiable
### ==============================================================
ds['T'] = pd.date_range(start='2007-01-01', end='2010-12-31', freq='1d')

### ==============================================================
### 4. calculate mean phosphorus concentration for each month
###    and for each year
### ==============================================================
phos_2007 = ds['cumulative'].where((ds['T.year']==2007),
                       drop=True).groupby('T.month').mean('T').values
phos_2008 = ds['cumulative'].where((ds['T.year']==2008),
                       drop=True).groupby('T.month').mean('T').values
phos_2009 = ds['cumulative'].where((ds['T.year']==2009),
                       drop=True).groupby('T.month').mean('T').values
phos_2010 = ds['cumulative'].where((ds['T.year']==2010),
                       drop=True).groupby('T.month').mean('T').values

### ==============================================================
### 5. Stack all the years
###    This creates an output with shape [12, X, Y]
### ==============================================================
tmp = np.stack((phos_2007, phos_2008, phos_2009, phos_2010))

### ==============================================================
### 6. Store in dataset
### ==============================================================
ds_out = xr.Dataset(
    {
    'phos_monthly': (['year', 'months', 'Y', 'X'],  tmp[:,4:8 :, :])
    },

    coords={
     'year': (['year'], [2007,2008,2009,2010]),
     'months': (['months'], ['may', 'june', 'july','august']),
     'Y': (['Y'], ds_grid['Y'].values),
     'X': (['X'], ds_grid['X'].values)
    }
    )

### ===================================
### 6.1 Display data
### ===================================
ds_out

<xarray.Dataset>
Dimensions:       (X: 200, Y: 276, months: 4, year: 4)
Coordinates:
  * year          (year) int64 2007 2008 2009 2010
  * months        (months) <U6 'may' 'june' 'july' 'august'
  * Y             (Y) float64 41.57 41.59 41.6 41.62 41.64 41.65 41.67 41.69 ...
  * X             (X) float64 -88.09 -88.07 -88.06 -88.04 -88.02 -88.01 ...
Data variables:
    phos_monthly  (year, months, Y, X) float32 0.0 0.0 0.0 0.0 0.0 0.0 0.0 ...

In [26]:
### ==============================================================
### 7. Save to netcdf
### ==============================================================
dir_clean='/Users/gloege/Documents/Projects/MITgcm-Michigan-Phosphorus/data/clean/'
ds_out.to_netcdf(f'{dir_clean}phosphorus_monthly_avg_MJJA_2007-2010.nc')

### ==============================================================
### 7.1 memory management
### ==============================================================
del ds, ds_grid, ds_out, tmp
del phos_2007, phos_2008, phos_2009, phos_2010

# 8. Calculate P load

In [27]:
### ===================================
### Get Loading
### ===================================
ploadDir = '/Users/gloege/Documents/Projects/MITgcm-Michigan-Phosphorus/data/raw/'
df_wrtds = pd.read_csv(f'{ploadDir}wrtds_pload_2007_2010_MTyr.csv', index_col='river')
df_wrtds = df_wrtds.rename({'Pere Marquette':'Pere_Marquette', 'St Joseph':'St_Joseph'})
load = df_wrtds.mean(axis=1)
print(load)

river
Milwaukee          77.25
Sheboygan          76.25
Fox               409.00
Menominee          60.75
Manistique         23.25
Manistee           37.50
Pere_Marquette     26.00
Muskegon           58.00
Grand             526.25
Kalamazoo         188.00
St_Joseph         421.50
dtype: float64
